TITLE: CAR SALES ANALYSIS

Introduction:

Within this project is a csv file containing over 50,0000 entries. Each entry contains various information regarding a car sale.
In my analysis, I focused on price, odometer mileage, manufacturer, and condition.

In [32]:
import pandas as pd
import streamlit as st
import plotly.express as px

df = pd.read_csv("vehicles_us.csv")
df = df.sort_values(by=['price'], ascending=False)

df.info()
display(df)

print("The highest sale price is", df['price'].max())
print("The lowest sale price is", df['price'].min())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51525 entries, 12504 to 25972
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.5+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
12504,375000,1999.0,nissan frontier,good,6.0,gas,115000.0,automatic,pickup,blue,1.0,2018-05-19,21
11359,300000,2015.0,ram 2500,excellent,NaN,diesel,NaN,automatic,truck,grey,1.0,2018-10-15,39
34389,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2019-02-02,28
1309,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2019-03-02,56
27375,189000,2014.0,ford f-150,good,6.0,gas,151248.0,automatic,truck,black,NaN,2018-09-25,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24988,1,NaN,chevrolet tahoe,like new,8.0,gas,NaN,automatic,SUV,blue,NaN,2019-02-10,81
11738,1,2015.0,ford edge,excellent,6.0,gas,NaN,automatic,SUV,black,1.0,2018-07-12,24
11739,1,2018.0,nissan altima,excellent,10.0,gas,42007.0,other,sedan,custom,1.0,2018-10-27,42
24989,1,2007.0,chevrolet tahoe,like new,8.0,gas,1.0,automatic,SUV,blue,NaN,2019-03-04,44


The highest sale price is 375000
The lowest sale price is 1


A sale price of $1.00 doesn't make any sense. Either the car was donated or that just means the data is missing.
I want to compare Condition vs Price and Model Year vs Price. In fact, any value under $1,000 doesn't make sense and isn't worth analyzing. 
It might be better to remove any price under $1,000. I will filter out all rows contaning a sale price of $1,000. Also, I will also remove any sales above $80,000.

I'm going to change model_year to int class type for less memory and replace missing values with 0.

I'm going to change odomoter to int class type for less memory. Also, any missing values, I will change to a value of -1. This makes more sense then putting 0. I wouldn't want any one else to believe the odometer is zero. I want it to be clear the value is missing.

There are missing values in is_4wd. A value of 1 should mean, the car is indeed Four Wheel Drive. So instead of having missing values, I will fill in missing values with 0, to represent vehicles without Four Wheel Drive.

Likewise, I will replace missing paint_color values with "unknown"

I will change date_posted in to DateTime data type.

In [36]:
#create new column for just the manufacturer
df['manufacturer'] = df['model'].apply(lambda x: x.split()[0])

#fill missing values with the median year. and change column to int type
year_avg = df['model_year'].median()
df['model_year'] = df['model_year'].fillna(year_avg)
df['model_year'] = df['model_year'].astype('Int64')

#fill in missing values for cylinders columns with most median cylinder type
cylinders_mode =  df['cylinders'].median()
df['cylinders'] = df['cylinders'].fillna(cylinders_mode)
df['cylinders'] = df['cylinders'].astype('Int64')

#fill missing values with the meadian odometer reading. and change column to int type
mileage_avg = df['odometer'].median()
df['odometer'] = df['odometer'].fillna(mileage_avg)
df['odometer'] = df['odometer'].astype('Int64')

#fill missing values with 0. and change column to int type
df['is_4wd'] = df['is_4wd'].fillna(0)
df['is_4wd'] = df['is_4wd'].astype('Int64')

#fill missing values with unknown
df['paint_color'] = df['paint_color'].fillna('unknown')

#fill missing values with unknown
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d')

#filter out all sales of $1
df = df.query("price > 1000  & price <= 80000")

df.info()
display(df)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 49944 entries, 1326 to 38576
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         49944 non-null  int64         
 1   model_year    49944 non-null  Int64         
 2   model         49944 non-null  object        
 3   condition     49944 non-null  object        
 4   cylinders     49944 non-null  Int64         
 5   fuel          49944 non-null  object        
 6   odometer      49944 non-null  Int64         
 7   transmission  49944 non-null  object        
 8   type          49944 non-null  object        
 9   paint_color   49944 non-null  object        
 10  is_4wd        49944 non-null  Int64         
 11  date_posted   49944 non-null  datetime64[ns]
 12  days_listed   49944 non-null  int64         
 13  manufacturer  49944 non-null  object        
dtypes: Int64(4), datetime64[ns](1), int64(2), object(7)
memory usage: 5.9+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
1326,79900,2011,ford f350,like new,8,diesel,10060,automatic,pickup,white,1,2018-07-16,22,ford
12485,78000,2009,honda odyssey,good,6,gas,126567,automatic,van,blue,0,2018-11-14,28,honda
50664,76995,2018,cadillac escalade,excellent,8,gas,4500,other,SUV,white,1,2018-10-25,21,cadillac
23774,76500,2015,chevrolet silverado 2500hd,like new,6,diesel,8989,automatic,pickup,white,1,2019-02-23,31,chevrolet
2790,76000,2016,chevrolet corvette,new,8,gas,1700,automatic,convertible,white,0,2018-07-15,79,chevrolet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21593,1099,2001,acura tl,fair,6,gas,226000,automatic,sedan,unknown,0,2019-03-06,28,acura
32314,1099,2000,hyundai elantra,fair,4,gas,151000,automatic,sedan,green,0,2018-08-27,66,hyundai
17543,1095,1998,ford explorer,fair,6,gas,140000,automatic,SUV,blue,0,2019-04-04,103,ford
12760,1050,1995,toyota corolla,fair,6,gas,428000,manual,sedan,black,0,2018-06-13,12,toyota


In [34]:
#scatterplot of mileage vs price
st.header('Mileage Vs Price')
#removing any outliers above 450,000 miles
mileage_mask = df['odometer'] < 450000
df_mileage = df[mileage_mask]
scatterplot = px.scatter(df_mileage, x='odometer', y='price')
st.write(scatterplot)


2023-01-27 14:11:55.553 
  command:

    streamlit run /Users/ggarces/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


Data Anaylsis 1: Mileage Vs Price
I simply chart (using a scatter plot) price vs mileage (odometer). I removed any outliers with mileage over 450,000 miles.
My hypothesis is intuitive; the more mileage, the less the car is worth.
Observing the chart, my hypothesis is correct. There is a clear negative correlation.

In [38]:
#distribution of price based on manufacturer
st.header('Compare price distribution based on manufacturer')
# get a list of car manufacturers
manufac_list = sorted(df['manufacturer'].unique())
# get user's inputs from a dropdown menu
manufacturer = st.selectbox(
                              label='Select manufacturer', # title of the select box
                              options=manufac_list, # options listed in the select box
                              index=manufac_list.index('chevrolet') # default pre-selected option
                              )

mask_filter = (df['manufacturer'] == manufacturer)

# add a checkbox if a user wants to pick all manufacturers
choose_all= st.checkbox('All Manufacturers', value=True)
if choose_all:
    df_filtered = df
else: 
    df_filtered = df[mask_filter] #filter the dataframe if unchecked

# create a plotly histogram figure
fig4 = px.histogram(df_filtered,
                      x='price',
                      nbins=30,
                      color='condition',
                      barmode='overlay')
# display the figure with streamlit
st.write(fig4)

Data Anaylsis 2: Price Distribution
I want to see to the price distribution between manufacturers and condition. I created a dropbox to choose each unique manufacturer. Users can also check the checkbox to see the price distribution between ALL manufacturers. Likewise, users can toggle on/off each condition to see the price distribution between conditions.